In [20]:
import obspy
from obspy.io.segy.core import _read_segy

filename = '/home/dnlab/Data-B/1001-1122/1511994323_1018_50500_16600_20171202_220556_632.sgy'
stream = _read_segy(filename, headonly=True)
print(stream.traces[0].stats)


         network: 
         station: 
        location: 
         channel: 
       starttime: 2017-11-29T22:25:23.000000Z
         endtime: 2017-11-29T22:25:39.000000Z
   sampling_rate: 1000.0
           delta: 0.001
            npts: 16001
           calib: 1.0
            segy: AttribDict({'trace_header': LazyTraceHeaderAttribDict({'unpacked_header': b'\xfa\xff\xff\xff\xfa\xff\xff\xff\xfa\x03\x00\x00\xfa\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x07\x00\x04\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\xccB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x80\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xff\xf6\xffM_3\x00\x15\x8a\x8a\x02\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x81>\xe8\x03\x03\x00\x00\x00\x00\x00\x01\x00(\x00H\x03\xe0.\x01\x00,\x01,\x01\x04\x00\x9d\x01\x06\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe1\x07M\x01\x16\x00\x19\x00\x17\x00\x04\x00 }\r

In [31]:
import matplotlib.pyplot as plt
import obspy
import numpy as np

# Load the SEGY file
filename = '/home/dnlab/Data-B/1001-1122/1511994323_1018_50500_16600_20171202_220556_632.sgy'
stream = obspy.read(filename)

# Extract the data and trace header information
data = np.array([trace.data for trace in stream.traces])
npts = stream.traces[0].stats.npts
sampling_rate = stream.traces[0].stats.sampling_rate
starttime = stream.traces[0].stats.starttime

print(data)
print(npts)
print(sampling_rate)
print(starttime)



[[-8.6211255e-03 -8.6097559e-03 -8.6101312e-03 ... -8.6270012e-03
  -8.6304834e-03 -8.6225774e-03]
 [ 4.7923021e+00  1.0675242e+01  9.4782658e+00 ... -1.0837941e-01
   1.0161659e-01 -8.5060950e-03]
 [ 2.6262314e-03  3.3074068e-03  3.3874351e-03 ...  3.0328280e-03
   2.8129078e-03  2.5832215e-03]
 ...
 [ 2.5407210e-06  2.5832412e-06  2.8842585e-06 ...  7.2876214e-06
   7.6792285e-06  6.2379572e-06]
 [ 3.0551371e-06  2.7638655e-06  2.7422345e-06 ...  5.0419499e-06
   4.1835178e-06  4.5364277e-06]
 [-1.5085160e-06 -2.3389075e-06 -3.2011690e-06 ...  3.6909471e-06
   2.0183377e-06  9.0470803e-07]]
16001
1000.0
2017-11-29T22:25:23.000000Z


In [1]:
from numpy.random import seed
from keras.callbacks import ModelCheckpoint
from keras import backend as keras
from utils import generate_data, DataGenerator
from unet import *
import flwr as fl

params = {'batch_size':1,
          'dim':(128,128,128),
          'n_channels':1,
          'shuffle': True}
seismPathT = "../../data_seismic/fl_test/client1/train/seis/"
faultPathT = "../../data_seismic/fl_test/client1/train/fault/"
seismPathV = "../../data_seismic/fl_test/client1/validation/seis/"
faultPathV = "../../data_seismic/fl_test/client1/validation/fault/"
train_ID = range(100)
valid_ID = range(10)
x_train, y_train = generate_data(dpath=seismPathT, fpath=faultPathT, data_IDs=train_ID, **params)
x_test, y_test = generate_data(dpath=seismPathV, fpath=faultPathV, data_IDs=valid_ID, **params)

print(len(x_train))
print('---------------------------')

train_generator = DataGenerator(dpath=seismPathT,fpath=faultPathT,
                                data_IDs=train_ID,**params)
valid_generator = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                data_IDs=valid_ID,**params)
print(train_generator)
print(len(train_generator[0]))

print("Finish loading dataset")
model = unet(input_size=(None, None, None,1))
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
              metrics=['accuracy'])

2023-05-16 12:24:57.671599: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 12:24:57.915531: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 12:24:58.889110: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-16 12:24:58.889208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

2
---------------------------
100
2
Finish loading dataset


2023-05-16 12:25:02.474513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5654 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:73:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d (Conv3D)                (None, None, None,   448         ['input_1[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_1 (Conv3D)              (None, None, None,   6928        ['conv3d[0][0]']                 
                                None, 16)                                                     

/home/dnlab/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [2]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100


2023-05-16 12:25:13.569285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-05-16 12:25:18.414927: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 272.08MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1/1 [==============================] - 14s 14s/step - loss: 0.7223 - accuracy: 0.0869
Epoch 2/100
1/1 [==============================] - 1s 838ms/step - loss: 0.7111 - accuracy: 0.1568
Epoch 3/100
1/1 [==============================] - 1s 840ms/step - loss: 0.7020 - accuracy: 0.3148
Epoch 4/100
1/1 [==============================] - 1s 839ms/step - loss: 0.6927 - accuracy: 0.5510
Epoch 5/100
1/1 [==============================] - 1s 839ms/step - loss: 0.6813 - accuracy: 0.7772
Epoch 6/100
1/1 [==============================] - 1s 839ms/step - loss: 0.6659 - accuracy: 0.8785
Epoch 7/100
1/1 [==============================] - 1s 839ms/step - loss: 0.6430 - accuracy: 0.9062
Epoch 8/100
1/1 [==============================] - 1s 839ms/step - loss: 0.6090 - accuracy: 0.9143
Epoch 9/100
1/1 [==============================] - 1s 834ms/step - loss: 0.5605 - accuracy: 0.9172
Epoch 10/100
1/1 [==============================] - 1s 838ms/step - loss: 0.4901 - accuracy: 0.9193
Epoch 11/100
1/1 [====